In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("../new_data.csv")
df.English = df.English.str.replace('\n', ' ')
df=df.dropna(axis=0)
df.head()

,Id,Enunciado,Tópico,Contexto,English
0,994,Meu primeiro programa\nEscreva um programa que...,Ambientação,Outros,"""My first show Write a program that prints the..."
1,996,Impressão de caracteres na tela\nEscreva um pr...,Ambientação,Outros,Screen character printing Write a program tha...
2,999,Impressão de caracteres na tela (Bart Simpson)...,Ambientação,Outros,Screen Character Printing (Bart Simpson) Writ...
3,1000,Operadores aritméticos\nQual o valor de X para...,Ambientação,Matemático,Arithmetic Operators What is the value of X s...
4,578,Operadores aritméticos \nQual o valor de Y par...,Ambientação,Matemático,Arithmetic Operators What is the value of Y s...


In [3]:
contador=0
lista=[]
while(contador<len(df['English'])):
    if(df['Contexto'][contador]=='Outros'):
        lista+=[contador]
    contador+=1
df=df.drop(lista)

In [4]:
import spacy
nlp = spacy.load('en')

In [5]:
master=[]
for elemento in df['English']:
    doc=nlp(elemento)
    lista=[]
    for token in doc:
        if(token.text not in nlp.Defaults.stop_words and token.lemma_ not in nlp.Defaults.stop_words and token.text.isalpha()):
            lista+=[token.lemma_]
    master+=[' '.join(lista)]

In [6]:
df['tokens']=master

In [25]:
X = df['tokens'].values
y = df['Contexto'].values

In [26]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [27]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y)

LabelEncoder()

In [28]:
from sklearn.model_selection import StratifiedKFold
vetor_X_train=[]
vetor_y_train=[]
vetor_X_test=[]
vetor_y_test=[]
split = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
for train_index, val_index in split.split(X, y):
    vetor_X_train+=[X[train_index]]
    vetor_X_test+=[X[val_index]]
    vetor_y_train+=[y[train_index]]
    vetor_y_test+=[y[val_index]]

/home/samuel/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


In [29]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer                    
from keras.preprocessing.sequence import pad_sequences
from keras import layers

In [30]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X)

vocab_size = len(tokenizer.word_index) + 1   
maxlen = 300
embedding_dim = 100
vocab_size

3546

In [31]:
def baseline_model():
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=300))
    
    model.add(layers.Conv1D(128, 64, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    
#     model.add(layers.Conv1D(128, 10, activation='relu'))
#     model.add(layers.GlobalMaxPooling1D())
    
#     model.add(layers.Flatten())
    
    model.add(layers.Dense(64, activation='relu'))
    model.add(Dropout(p=0.1))
    model.add(layers.Dense(23, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [32]:
contador=0
predictions=[]
while(contador<len(vetor_X_train)):
    X_train=vetor_X_train[contador]
    y_train=vetor_y_train[contador]
    X_test=vetor_X_test[contador]
    y_test=vetor_y_test[contador]
    
    encoded_y = encoder.transform(y_train)
    # convert integers to dummy variables (i.e. one hot encoded)
    y_train = np_utils.to_categorical(encoded_y)
        
#     X_train = vectorizer.fit_transform(X_train)
    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)
    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
    
    estimator = KerasClassifier(build_fn=baseline_model, epochs=25, batch_size=20)
    estimator.fit(X_train, y_train)
#     y_pred = estimator.predict(vectorizer.fit_transform(X_test))
    y_pred = estimator.predict(X_test)
    predictions+=[y_pred]

    contador+=1

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/home/samuel/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
887/887 [==============================] - 13s 15ms/step - loss: 0.2722 - accuracy: 0.9102
Epoch 2/25
887/887 [==============================] - 13s 15ms/step - loss: 0.1597 - accuracy: 0.9551
Epoch 3/25
887/887 [==============================] - 12s 14ms/step - loss: 0.1316 - accuracy: 0.9623
Epoch 4/25
887/887 [==============================] - 11s 12ms/step - loss: 0.0874 - accuracy: 0.9745
Epoch 5/25
887/887 [==============================] - 11s 13ms/step - loss: 0.0510 - accuracy: 0.9853
Epoch 6/25
887/887 [==============================] - 10s 12ms/step - loss: 0.0289 - accuracy: 0.9924
Epoch 7/25
887/887 [==============================] - 10s 12ms/step - loss: 0.0173 - accuracy: 0.9956
Epoch 8/25
887/887 [==============================] - 10s 12ms/step - loss: 0.0106 - accuracy: 0.9973
Epoch 9/25
887/887 [==============================] - 10s 12ms/step - loss: 0.0073 - accuracy: 0.9982
Epoch 10/25
887/887 [==============================] - 10s 12ms/step - loss: 0.005

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/home/samuel/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
887/887 [==============================] - 10s 12ms/step - loss: 0.2757 - accuracy: 0.9027
Epoch 2/25
887/887 [==============================] - 10s 12ms/step - loss: 0.1592 - accuracy: 0.9560
Epoch 3/25
887/887 [==============================] - 10s 12ms/step - loss: 0.1232 - accuracy: 0.9659
Epoch 4/25
887/887 [==============================] - 10s 12ms/step - loss: 0.0800 - accuracy: 0.9769
Epoch 5/25
887/887 [==============================] - 10s 12ms/step - loss: 0.0426 - accuracy: 0.9877
Epoch 6/25
887/887 [==============================] - 10s 12ms/step - loss: 0.0235 - accuracy: 0.9936
Epoch 7/25
887/887 [==============================] - 10s 12ms/step - loss: 0.0128 - accuracy: 0.9969
Epoch 8/25
887/887 [==============================] - 10s 12ms/step - loss: 0.0085 - accuracy: 0.9983
Epoch 9/25
887/887 [==============================] - 10s 12ms/step - loss: 0.0063 - accuracy: 0.9986
Epoch 10/25
887/887 [==============================] - 10s 12ms/step - loss: 0.004

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/home/samuel/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

Epoch 1/25
888/888 [==============================] - 10s 12ms/step - loss: 0.3235 - accuracy: 0.8824
Epoch 2/25
888/888 [==============================] - 10s 11ms/step - loss: 0.1629 - accuracy: 0.9548
Epoch 3/25
888/888 [==============================] - 10s 12ms/step - loss: 0.1260 - accuracy: 0.9637
Epoch 4/25
888/888 [==============================] - 10s 11ms/step - loss: 0.0865 - accuracy: 0.9764
Epoch 5/25
888/888 [==============================] - 10s 11ms/step - loss: 0.0501 - accuracy: 0.9853
Epoch 6/25
888/888 [==============================] - 10s 11ms/step - loss: 0.0263 - accuracy: 0.9926
Epoch 7/25
888/888 [==============================] - 10s 11ms/step - loss: 0.0144 - accuracy: 0.9962
Epoch 8/25
888/888 [==============================] - 10s 11ms/step - loss: 0.0089 - accuracy: 0.9977
Epoch 9/25
888/888 [==============================] - 10s 11ms/step - loss: 0.0066 - accuracy: 0.9985
Epoch 10/25
888/888 [==============================] - 10s 11ms/step - loss: 0.004

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/home/samuel/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
888/888 [==============================] - 10s 12ms/step - loss: 0.2642 - accuracy: 0.9150
Epoch 2/25
888/888 [==============================] - 10s 11ms/step - loss: 0.1581 - accuracy: 0.9552
Epoch 3/25
888/888 [==============================] - 10s 11ms/step - loss: 0.1278 - accuracy: 0.9633
Epoch 4/25
888/888 [==============================] - 10s 11ms/step - loss: 0.0877 - accuracy: 0.9751
Epoch 5/25
888/888 [==============================] - 10s 12ms/step - loss: 0.0491 - accuracy: 0.9862
Epoch 6/25
888/888 [==============================] - 12s 14ms/step - loss: 0.0261 - accuracy: 0.9938
Epoch 7/25
888/888 [==============================] - 11s 12ms/step - loss: 0.0156 - accuracy: 0.9961
Epoch 8/25
888/888 [==============================] - 11s 12ms/step - loss: 0.0103 - accuracy: 0.9977
Epoch 9/25
888/888 [==============================] - 11s 12ms/step - loss: 0.0063 - accuracy: 0.9987
Epoch 10/25
888/888 [==============================] - 10s 12ms/step - loss: 0.004

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/home/samuel/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
888/888 [==============================] - 10s 11ms/step - loss: 0.2833 - accuracy: 0.9018
Epoch 2/25
888/888 [==============================] - 10s 11ms/step - loss: 0.1592 - accuracy: 0.9572
Epoch 3/25
888/888 [==============================] - 10s 11ms/step - loss: 0.1272 - accuracy: 0.9653
Epoch 4/25
888/888 [==============================] - 10s 11ms/step - loss: 0.0858 - accuracy: 0.9748
Epoch 5/25
888/888 [==============================] - 10s 11ms/step - loss: 0.0527 - accuracy: 0.9846
Epoch 6/25
888/888 [==============================] - 10s 11ms/step - loss: 0.0323 - accuracy: 0.9912
Epoch 7/25
888/888 [==============================] - 10s 11ms/step - loss: 0.0206 - accuracy: 0.9948
Epoch 8/25
888/888 [==============================] - 10s 11ms/step - loss: 0.0129 - accuracy: 0.9968
Epoch 9/25
888/888 [==============================] - 10s 11ms/step - loss: 0.0095 - accuracy: 0.9979
Epoch 10/25
888/888 [==============================] - 10s 11ms/step - loss: 0.007

In [33]:
y_pred=[]
for lista1 in predictions:
    for elemento1 in encoder.inverse_transform(lista1):
        y_pred+=[elemento1]


In [34]:
y_test_sla=[]
for lista2 in vetor_y_test:
    for elemento2 in lista2:
        y_test_sla+=[elemento2]

In [35]:
from sklearn import metrics

In [36]:
print(metrics.confusion_matrix(y_test_sla,y_pred))

[[ 31   2   0   0   0   0   0   0   0   0   0   1   1   0   0   0   0   0
    0   0   0   0   0]
 [  2 108   0   0   1   1   0   0   0   0   1   2   3   0   0   0   0   0
    0   2   0   0   0]
 [  0   0   5   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0
    0   0   0   0   0]
 [  0   2   0  13   0   0   0   0   0   0   0   1   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   1   0   0  14   0   0   0   0   0   0   1   4   0   0   0   0   0
    0   0   0   0   1]
 [  0   1   0   0   0  71   1   0   0   2   0   0   3   0   0   0   0   0
    0   0   0   0   1]
 [  0   0   0   0   0   0  42   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0   0  27   0   0   0   0   3   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  31   0   0   1   3   0   0   0   0   0
    0   0   0   0   1]
 [  0   1   0   0   0   1   0   0   0   7   0   0   2   0   0   0   0   0
    0   0   0   0   0]
 [  0   1   0   0   0   0   0 

In [37]:
print(metrics.classification_report(y_test_sla,y_pred))

                 precision    recall  f1-score   support

       Bancário       0.94      0.89      0.91        35
      Comercial       0.81      0.90      0.85       120
  Computacional       1.00      0.83      0.91         6
        Consumo       1.00      0.81      0.90        16
    Data e hora       0.93      0.67      0.78        21
        Escolar       0.93      0.90      0.92        79
        Esporte       0.95      1.00      0.98        42
Filmes e séries       1.00      0.90      0.95        30
         Física       0.94      0.86      0.90        36
      Geografia       0.70      0.64      0.67        11
        Imposto       0.80      0.80      0.80         5
          Jogos       0.92      0.90      0.91        96
     Matemático       0.89      1.00      0.94       261
  Meio ambiente       1.00      0.89      0.94        18
       Pesquisa       1.00      0.89      0.94        18
         Pessoa       0.95      0.86      0.90        22
      População       1.00    

In [38]:
print(metrics.accuracy_score(y_test_sla,y_pred))

0.9087221095334685


In [39]:
np.savetxt('ytrue_CNN_Context-Others', y_test_sla, fmt='%s')
np.savetxt('ypred_CNN_Context-Others', y_pred, fmt='%s')

In [43]:
def noOthers(df):
    contador=0
    lista=[]
    while(contador<len(df['English'])):
        if(df['Contexto'][contador]=='Outros'):
            lista+=[contador]
        contador+=1
    return df.drop(lista)

In [45]:
dfs_train = []
dfs_test = []
for i in range(0, 10):
    df_train = pd.read_csv('../dadosParafrase/treino'+str(i+1)+'.csv')
    df_train=noOthers(df_train)
    dfs_train.append(df_train)
    df_test = pd.read_csv('../dadosParafrase/teste'+str(i+1)+'.csv')
    df_test=noOthers(df_test)
    dfs_test.append(df_test)
    

In [47]:
contador=0
predictions=[]
vetor_y_test=[]
while(contador<len(dfs_train)):
    X_train=dfs_train[contador]['master'].values
    y_train=dfs_train[contador]['Contexto'].values
    X_test=dfs_test[contador]['master'].values
    vetor_y_test+=[dfs_test[contador]['Contexto'].values]
    
    
    encoded_y = encoder.transform(y_train)
    # convert integers to dummy variables (i.e. one hot encoded)
    y_train = np_utils.to_categorical(encoded_y)
        
#     X_train = vectorizer.fit_transform(X_train)
    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)
    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
    
    estimator = KerasClassifier(build_fn=baseline_model, epochs=25, batch_size=20)
    estimator.fit(X_train, y_train)
#     y_pred = estimator.predict(vectorizer.fit_transform(X_test))
    y_pred = estimator.predict(X_test)
    predictions+=[y_pred]

    contador+=1

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/home/samuel/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
2870/2870 [==============================] - 31s 11ms/step - loss: 0.1943 - accuracy: 0.9434
Epoch 2/25
2870/2870 [==============================] - 34s 12ms/step - loss: 0.0487 - accuracy: 0.9849
Epoch 3/25
2870/2870 [==============================] - 33s 12ms/step - loss: 0.0106 - accuracy: 0.9973
Epoch 4/25
2870/2870 [==============================] - 33s 12ms/step - loss: 0.0042 - accuracy: 0.9991
Epoch 5/25
2870/2870 [==============================] - 33s 12ms/step - loss: 0.0022 - accuracy: 0.9996
Epoch 6/25
2870/2870 [==============================] - 33s 12ms/step - loss: 0.0014 - accuracy: 0.9997
Epoch 7/25
2870/2870 [==============================] - 33s 12ms/step - loss: 0.0011 - accuracy: 0.9998
Epoch 8/25
2870/2870 [==============================] - 33s 12ms/step - loss: 7.5076e-04 - accuracy: 0.9999
Epoch 9/25
2870/2870 [==============================] - 33s 12ms/step - loss: 7.1566e-04 - accuracy: 0.9999
Epoch 10/25
2870/2870 [==============================] -

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/home/samuel/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
2875/2875 [==============================] - 34s 12ms/step - loss: 0.1930 - accuracy: 0.9481
Epoch 2/25
2875/2875 [==============================] - 34s 12ms/step - loss: 0.0553 - accuracy: 0.9834
Epoch 3/25
2875/2875 [==============================] - 34s 12ms/step - loss: 0.0105 - accuracy: 0.9975
Epoch 4/25
2875/2875 [==============================] - 34s 12ms/step - loss: 0.0037 - accuracy: 0.9992
Epoch 5/25
2875/2875 [==============================] - 34s 12ms/step - loss: 0.0019 - accuracy: 0.9997
Epoch 6/25
2875/2875 [==============================] - 34s 12ms/step - loss: 0.0016 - accuracy: 0.9998
Epoch 7/25
2875/2875 [==============================] - 34s 12ms/step - loss: 9.4971e-04 - accuracy: 0.9998
Epoch 8/25
2875/2875 [==============================] - 34s 12ms/step - loss: 0.0010 - accuracy: 0.9998
Epoch 9/25
2875/2875 [==============================] - 35s 12ms/step - loss: 8.7971e-04 - accuracy: 0.9999
Epoch 10/25
2875/2875 [==============================] -

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/home/samuel/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

Epoch 1/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.2071 - accuracy: 0.9382
Epoch 2/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0581 - accuracy: 0.9819
Epoch 3/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0119 - accuracy: 0.9970
Epoch 4/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0044 - accuracy: 0.9992
Epoch 5/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0025 - accuracy: 0.9995
Epoch 6/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0015 - accuracy: 0.9998
Epoch 7/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0014 - accuracy: 0.9997
Epoch 8/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0013 - accuracy: 0.9998
Epoch 9/25
2875/2875 [==============================] - 31s 11ms/step - loss: 8.2159e-04 - accuracy: 0.9998
Epoch 10/25
2875/2875 [==============================] - 31s

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/home/samuel/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.2010 - accuracy: 0.9403
Epoch 2/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0572 - accuracy: 0.9821
Epoch 3/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0117 - accuracy: 0.9970
Epoch 4/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0045 - accuracy: 0.9990
Epoch 5/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0023 - accuracy: 0.9996
Epoch 6/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0015 - accuracy: 0.9997
Epoch 7/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0010 - accuracy: 0.9999
Epoch 8/25
2875/2875 [==============================] - 31s 11ms/step - loss: 8.0051e-04 - accuracy: 0.9999
Epoch 9/25
2875/2875 [==============================] - 31s 11ms/step - loss: 0.0011 - accuracy: 0.9998
Epoch 10/25
2875/2875 [==============================] - 31s

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/home/samuel/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
2866/2866 [==============================] - 31s 11ms/step - loss: 0.1944 - accuracy: 0.9447
Epoch 2/25
2866/2866 [==============================] - 32s 11ms/step - loss: 0.0491 - accuracy: 0.9852
Epoch 3/25
2866/2866 [==============================] - 31s 11ms/step - loss: 0.0103 - accuracy: 0.9974
Epoch 4/25
2866/2866 [==============================] - 31s 11ms/step - loss: 0.0047 - accuracy: 0.9988
Epoch 5/25
2866/2866 [==============================] - 31s 11ms/step - loss: 0.0027 - accuracy: 0.9994
Epoch 6/25
2866/2866 [==============================] - 31s 11ms/step - loss: 0.0016 - accuracy: 0.9997
Epoch 7/25
2866/2866 [==============================] - 31s 11ms/step - loss: 0.0014 - accuracy: 0.9998
Epoch 8/25
2866/2866 [==============================] - 31s 11ms/step - loss: 0.0011 - accuracy: 0.9998
Epoch 9/25
2866/2866 [==============================] - 31s 11ms/step - loss: 7.8533e-04 - accuracy: 0.9998
Epoch 10/25
2866/2866 [==============================] - 31s

In [48]:
y_pred=[]
for lista1 in predictions:
    for elemento1 in encoder.inverse_transform(lista1):
        y_pred+=[elemento1]


In [49]:
y_test_sla=[]
for lista2 in vetor_y_test:
    for elemento2 in lista2:
        y_test_sla+=[elemento2]

In [50]:
from sklearn import metrics

In [51]:
print(metrics.confusion_matrix(y_test_sla,y_pred))

[[ 31   3   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0
    0   0   0   0   0]
 [  2 109   0   0   0   0   1   0   0   0   0   2   2   0   0   0   0   1
    0   1   1   0   1]
 [  0   0   6   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0  15   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   1]
 [  0   0   0   0  17   0   0   0   0   0   0   0   2   0   0   0   0   0
    0   0   1   0   1]
 [  0   0   0   0   0  75   0   0   0   1   0   0   2   0   0   1   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0  42   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   1   0  29   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  32   0   0   1   0   1   0   1   0   0
    0   0   0   0   1]
 [  0   0   0   0   1   0   0   0   0   9   0   0   1   0   0   0   0   0
    0   0   0   0   0]
 [  0   1   0   0   0   0   0 

In [52]:
print(metrics.classification_report(y_test_sla,y_pred))

                 precision    recall  f1-score   support

       Bancário       0.91      0.89      0.90        35
      Comercial       0.92      0.91      0.92       120
  Computacional       1.00      1.00      1.00         6
        Consumo       1.00      0.94      0.97        16
    Data e hora       0.81      0.81      0.81        21
        Escolar       0.96      0.95      0.96        79
        Esporte       0.98      1.00      0.99        42
Filmes e séries       0.97      0.97      0.97        30
         Física       0.97      0.89      0.93        36
      Geografia       0.82      0.82      0.82        11
        Imposto       0.80      0.80      0.80         5
          Jogos       0.91      0.92      0.91        96
     Matemático       0.95      0.99      0.97       261
  Meio ambiente       0.94      0.89      0.91        18
       Pesquisa       1.00      1.00      1.00        18
         Pessoa       0.86      0.86      0.86        22
      População       0.92    

In [53]:
print(metrics.accuracy_score(y_test_sla,y_pred))

0.9320486815415822


In [54]:
np.savetxt('ytrue_CNN_Context-Others+PAR', y_test_sla, fmt='%s')
np.savetxt('ypred_CNN_Context-Others+PAR', y_pred, fmt='%s')

In [61]:
def baseline_model():
    
    model = Sequential()
    model.add(Dense(64, activation='tanh', input_shape=(300,)))
    model.add(Dropout(p=0.1))
    model.add(Dense(64, activation='tanh'))
    model.add(Dropout(p=0.1))
    model.add(Dense(64, activation='tanh'))
    model.add(Dropout(p=0.1))
    model.add(Dense(7, activation='softmax'))


    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [62]:
estimator = KerasClassifier(build_fn=baseline_model)

In [63]:
from sklearn.model_selection import GridSearchCV
paramaters={'epochs':[150,200,250],
            'batch_size':[10,20],
            'optimizer':['adam','rmsprop']}

In [64]:
X = df['tokens'].values
X = vectorizer.fit_transform(X)
y = df['category'].values
# encoder = LabelEncoder()
# encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
y = np_utils.to_categorical(encoded_y)

Numer of samples with no words found: 0 / 4610


In [ ]:
grid=GridSearchCV(estimator=estimator,
                 param_grid=paramaters,
                 cv=10)
grid.fit(X,y)

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  """
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  import sys
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  if __name__ == '__main__':


Epoch 1/150
4149/4149 [==============================] - 1s 166us/step - loss: 1.4529 - accuracy: 0.4102
Epoch 2/150
4149/4149 [==============================] - 1s 142us/step - loss: 1.3049 - accuracy: 0.4784
Epoch 3/150
4149/4149 [==============================] - 1s 153us/step - loss: 1.2429 - accuracy: 0.5020
Epoch 4/150
4149/4149 [==============================] - 1s 146us/step - loss: 1.2135 - accuracy: 0.5170
Epoch 5/150
4149/4149 [==============================] - 1s 158us/step - loss: 1.1933 - accuracy: 0.5225
Epoch 6/150
4149/4149 [==============================] - 1s 145us/step - loss: 1.1670 - accuracy: 0.5396
Epoch 7/150
4149/4149 [==============================] - 1s 140us/step - loss: 1.1505 - accuracy: 0.5493
Epoch 8/150
4149/4149 [==============================] - 1s 133us/step - loss: 1.1513 - accuracy: 0.5466
Epoch 9/150
4149/4149 [==============================] - 1s 141us/step - loss: 1.1215 - accuracy: 0.5657
Epoch 10/150
4149/4149 [==============================]

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  """
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  import sys
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  if __name__ == '__main__':


Epoch 1/150
4149/4149 [==============================] - 1s 192us/step - loss: 1.4930 - accuracy: 0.4261
Epoch 2/150
4149/4149 [==============================] - 1s 168us/step - loss: 1.3089 - accuracy: 0.4992
Epoch 3/150
4149/4149 [==============================] - 1s 140us/step - loss: 1.2573 - accuracy: 0.5107
Epoch 4/150
4149/4149 [==============================] - 1s 179us/step - loss: 1.2265 - accuracy: 0.5271
Epoch 5/150
4149/4149 [==============================] - 1s 231us/step - loss: 1.2002 - accuracy: 0.5450
Epoch 6/150
4149/4149 [==============================] - 1s 225us/step - loss: 1.1726 - accuracy: 0.5490
Epoch 7/150
4149/4149 [==============================] - 1s 127us/step - loss: 1.1621 - accuracy: 0.5544
Epoch 8/150
4149/4149 [==============================] - 1s 153us/step - loss: 1.1455 - accuracy: 0.5592
Epoch 9/150
4149/4149 [==============================] - 0s 118us/step - loss: 1.1449 - accuracy: 0.5625
Epoch 10/150
4149/4149 [==============================]

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  """
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  import sys
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  if __name__ == '__main__':


...

4149/4149 [==============================] - 1s 182us/step - loss: 0.2177 - accuracy: 0.9231
Epoch 155/200
4149/4149 [==============================] - 1s 183us/step - loss: 0.2228 - accuracy: 0.9217
Epoch 156/200
4149/4149 [==============================] - 1s 166us/step - loss: 0.2198 - accuracy: 0.9272
Epoch 157/200
4149/4149 [==============================] - 1s 159us/step - loss: 0.2033 - accuracy: 0.9287
Epoch 158/200
4149/4149 [==============================] - 1s 157us/step - loss: 0.2245 - accuracy: 0.9171
Epoch 159/200
4149/4149 [==============================] - 1s 145us/step - loss: 0.2063 - accuracy: 0.9301
Epoch 160/200
4149/4149 [==============================] - 1s 155us/step - loss: 0.2080 - accuracy: 0.9289
Epoch 161/200
4149/4149 [==============================] - 1s 150us/step - loss: 0.2063 - accuracy: 0.9229
Epoch 162/200
4149/4149 [==============================] - 1s 153us/step - loss: 0.2075 - accuracy: 0.9301
Epoch 163/200
4149/4149 [==========================

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  """
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  import sys
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  if __name__ == '__main__':


Epoch 1/200
4149/4149 [==============================] - 3s 632us/step - loss: 1.3718 - accuracy: 0.4232
Epoch 2/200
4149/4149 [==============================] - 0s 115us/step - loss: 1.2293 - accuracy: 0.4796
Epoch 3/200
4149/4149 [==============================] - 0s 118us/step - loss: 1.1725 - accuracy: 0.5107
Epoch 4/200
4149/4149 [==============================] - 0s 119us/step - loss: 1.1349 - accuracy: 0.5300
Epoch 5/200
4149/4149 [==============================] - 0s 116us/step - loss: 1.1303 - accuracy: 0.5377
Epoch 6/200
4149/4149 [==============================] - 1s 121us/step - loss: 1.0942 - accuracy: 0.5546
Epoch 7/200
4149/4149 [==============================] - 1s 153us/step - loss: 1.0923 - accuracy: 0.5522
Epoch 8/200
4149/4149 [==============================] - 1s 132us/step - loss: 1.0754 - accuracy: 0.5536
Epoch 9/200
4149/4149 [==============================] - 1s 134us/step - loss: 1.0642 - accuracy: 0.5678
Epoch 10/200
4149/4149 [==============================]

4149/4149 [==============================] - 1s 154us/step - loss: 0.1984 - accuracy: 0.9262
Epoch 155/200
4149/4149 [==============================] - 1s 137us/step - loss: 0.1990 - accuracy: 0.9289
Epoch 156/200
4149/4149 [==============================] - 1s 156us/step - loss: 0.1779 - accuracy: 0.9383
Epoch 157/200
4149/4149 [==============================] - 1s 133us/step - loss: 0.1968 - accuracy: 0.9270
Epoch 158/200
4149/4149 [==============================] - 1s 151us/step - loss: 0.2022 - accuracy: 0.9299
Epoch 159/200
4149/4149 [==============================] - 1s 134us/step - loss: 0.2043 - accuracy: 0.9253
Epoch 160/200
4149/4149 [==============================] - 1s 163us/step - loss: 0.2068 - accuracy: 0.9315
Epoch 161/200
4149/4149 [==============================] - 1s 137us/step - loss: 0.1903 - accuracy: 0.9340
Epoch 162/200
4149/4149 [==============================] - 1s 152us/step - loss: 0.1981 - accuracy: 0.9287
Epoch 163/200
4149/4149 [==========================

/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  """
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  import sys
/home/samuel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  if __name__ == '__main__':


In [ ]:
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)